In [ ]:

def load_rnn():
    with open('model_RNN.pkl','rb') as f:  # Python 3: open(..., 'rb')
        _, model, _, _ = pickle.load(f)
    return model


def load_svc():
    with open('images_hz/model_SVC.pkl','rb') as f:
        model, _, _ = pickle.load(f)
    return model

def prediction(classifier):
    if classifier == 'RNN':
        clf = load_rnn()
    elif classifier == 'SVC':
        clf = load_svc()
    return clf

def scores(clf, user_input_word):
    return prediction(option).predict(user_input_word)


# use the st.selectbox() method to choose between the RandomForest classifier, the SVM classifier and the LogisticRegression classifier. Then return to the Streamlit web application to view the select box.
choice = ['RNN', 'SVC']
option = st.selectbox('Choice of the model', choice)
st.write('The chosen model is :', option)

user_input_word = st.text_input("Input a sentense: ", 'Merry Christmas!')
output_st = scores(user_input_word)
st.write('The input text is likely to be category :\n', output_st)